# Workplace interaction

Import some plotting libraries and set some defaults:

In [ ]:
import sys
import numpy as np
import pandas as pd
from IPython.display import display, Markdown
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

sns.set_context(context='paper', font_scale=0.8, rc=None)
# sns.set_context(context='notebook', font_scale=1, rc=None)

sns.set_style("ticks", {"axes.spines.top": False, 
                        "axes.spines.right": False,
                       "legend.edgecolor": 'white'})
import random
random_seed = 5

sys.path.append('..')
from global_params import load_params

In [ ]:
%%html
<style> table { font-variant-numeric: tabular-nums; } </style>

> **Tip:**  
> Producing figures in svg format (scalable vector graphics) makes for sharp plots on webpages. However, if you make plots with thousands of observations you should set this to `'png'` instead:

In [ ]:
%config InlineBackend.figure_formats = ['retina'] 

> **Tip:**  
> Some values apply globally to your analysis. E.g., sample sizes, cutoffs, names, rates, etc. Keeping those in a a yml file like `interaction_params.yml` and loading them in each notebook avoids the risk of manually adding/updating them in each notebook where they are used. You can use the `load_globals` function imported above from `global_params.py` to produce an object holding all the global values:

In [ ]:
params = load_params('../global_params.yml')
params

In [ ]:
params.sample_size

## Sampling

In [ ]:
subjects = pd.read_csv('../data/data_table.csv')
assert subjects.index.size == params.sample_size
subjects

> **Tip:**  
> By adding a label and caption to a cell displaying a table, you can refer to that table elsewhere and insert it in a manuscript.

In [ ]:
#| label: tbl-subjects
#| tbl-cap: "People included in the analysis."

# with pd.option_context('display.max_rows', None,): # prints *all* rows
#     display(subjects.style.hide()) # .style.hide() hides the index)
subjects.style.hide()

> **Tip:**  
> By generaing markdown for descriptions that will eventually end up in the manuscript, you can imbed python values. It also ensures that the manuscript exactly reflects the notebook.

In [ ]:
#| echo: false
#| label: doc-sampling

Markdown(f"""

The {params.sample_size} subjects from workplaces in Denmark were interviewed .... blah blah 
blah blah blah blah blah blah blah blah blah blah blah blah blah blah blah blah blah 
blah blah blah blah blah blah blah blah blah blah blah blah blah blah blah blah blah 
""")

## Interviews

In interviewed `{python} params.sample_size` workplace individuals were interviewed by .... blah, blah, blah, blah, blah, blah, blah, blah, blah, blah, blah, blah, blah, blah, 
blah, blah, blah, blah, blah, blah, blah, blah, blah, blah, blah, blah, blah, blah, 

In [ ]:
# generate some random sample data
scores = pd.DataFrame({
    'name': subjects.name, 
    'informality': np.random.normal(loc=10, scale=1, size=params.sample_size)
})
data = pd.merge(subjects, scores, on='name')
data


In [ ]:
#| label: fig-danishinteraction
#| fig-cap: "Figure legends are defined alongside the figure in the notebook. The figure size in the notebook is determines its size when embedded in a document 4x3 inches."

plt.figure(figsize=(4,3))
sns.scatterplot(x='age', y='informality', data=data, hue='position', palette='viridis')
plt.ylabel('How informal you can be')
plt.xlabel('Age')
plt.legend(title='Seniority', loc='lower right')
plt.ylim(bottom=0) ;

or plotted differently:

In [ ]:
#| label: fig-danishinteractionfacets
#| fig-cap: "Figure legends are defined alongside the figure in the notebook. The figure size in the notebook is determines its size when embedded in a document 4x3 inches."
#| column: page

#plt.figure(figsize=(4,3))
g = sns.FacetGrid(data=data, col="nationality",hue="position", palette='viridis', aspect=0.9)
g.map(sns.scatterplot, 'age', 'informality')
#sns.scatterplot(x='age', y='informality', data=data, hue='position', palette='viridis')
plt.ylabel('How informal you can be')
plt.xlabel('Age')
plt.legend(title='Seniority', loc='lower right') ;

Seems Danish people act very informally unaffected by age and seniority.

In [ ]:
informality_age_cor = data.informality.corr(data.age)
informality_age_cor

In [ ]:
mean_informality = data.groupby(['position', 'nationality']).informality.mean().to_frame().reset_index()
mean_informality = mean_informality.sort_values('informality')#.style.hide() # hide index

To show the `mean_informality` table, in a way that allow you to cite and embed it elsewhere, you need to provide a label, and preferably a caption, for it, as shown below. For this use, you also need to use the display function:

In [ ]:
#| label: tbl-mean-informality
#| tbl-cap: "Mean interaction scores by position and nationality."

display(mean_informality)

> **Tip:**
> To render a nice table without the index column, you can do this to the `display` function instead:
> `Markdown(df.to_markdown(index=False))`


> The table above shows the mean informality scores by

In [ ]:
#| echo: false
#| label: fig-meaninformality
#| fig-cap: "Figure legends are defined alongside the figure in the notebook. The figure size in the notebook is determines its size when embedded in a document 4x3 inches."

plt.figure(figsize=(4,3))
g = sns.pointplot(data=data, x='position', y='informality', hue='nationality')
sns.move_legend(g, loc="upper left", bbox_to_anchor=(1, 1), frameon=False)

In [ ]:
#| echo: false
#| label: doc-formality-correlation

Markdown(f"""
         
The correlation between informality and age was {round(informality_age_cor, 3)}.

""")

In [ ]:
#| label: fig-danish-interaction-regressions
#| fig-cap: "Figure legends are defined alongside the figure in the notebook. The figure size in the notebook is determines its size when embedded in a document 4x3 inches."

sns.lmplot(x='age', y='informality', data=data, hue='position', palette='viridis', height=3, aspect=4/3)
plt.ylabel('How informal you can be')
plt.xlabel('Age') ;